![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Use watsonx Granite Model Series, Chroma, and LangChain to answer questions (RAG)

#### Disclaimers

- Use only Projects and Spaces that are available in watsonx context.

## Notebook content
This notebook contains the steps and code to demonstrate support of Retrieval Augumented Generation in watsonx.ai. It introduces commands for data retrieval, knowledge base building & querying, and model testing.

Some familiarity with Python is helpful. This notebook uses Python 3.10.

### About Retrieval Augmented Generation
Retrieval Augmented Generation (RAG) is a versatile pattern that can unlock a number of use cases requiring factual recall of information, such as querying a knowledge base in natural language.

In its simplest form, RAG requires 3 steps:

- Index knowledge base passages (once)
- Retrieve relevant passage(s) from knowledge base (for every user query)
- Generate a response by feeding retrieved passage into a large language model (for every user query)

## Contents

This notebook contains the following parts:

- [Setup](#setup)
- [Document data loading](#data)
- [Build up knowledge base](#build_base)
- [Foundation Models on watsonx](#models)
- [Generate a retrieval-augmented response to a question](#predict)
- [Summary and next steps](#summary)


<a id="setup"></a>
##  Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Create a <a href="https://cloud.ibm.com/catalog/services/watson-machine-learning" target="_blank" rel="noopener no referrer">Watson Machine Learning (WML) Service</a> instance (a free plan is offered and information about how to create the instance can be found <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/getting-started/wml-plans.html?context=wx&audience=wdp" target="_blank" rel="noopener no referrer">here</a>).


### Install and import the dependecies

In [ ]:
# !pip install "langchain==0.1.10" | tail -n 1
# !pip install "ibm-watsonx-ai>=0.2.6" | tail -n 1
# !pip install -U langchain_ibm | tail -n 1
# !pip install wget | tail -n 1
# !pip install sentence-transformers | tail -n 1
# !pip install "chromadb" --upgrade | tail -n 1
# !pip install "sqlalchemy==2.0.1" | tail -n 1
# !pip install "lastmile-eval>=0.0.45"
# !pip install "lastmile-eval[ui]"
# !pip install "tracing-auto-instrumentation[langchain]" --upgrade

### watsonx API connection
This cell defines the credentials required to work with watsonx API for Foundation
Model inferencing.

**Action:** Provide the IBM Cloud user API key. For details, see <a href="https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui" target="_blank" rel="noopener no referrer">documentation</a>.

For Jupyter notebook, save these in an `.env` file within this project directory. For Google Colab, use Secret Manager to set these (key icon on the left).

In [ ]:
import os

try:
    # First try this in case we're running on Google Collab
    from google.colab import userdata
    os.environ['WATSONX_API_KEY'] =  userdata.get('WATSONX_API_KEY')
    os.environ['PROJECT_ID'] =  userdata.get('PROJECT_ID')
    os.environ['SPACE_ID'] =  userdata.get('SPACE_ID')
except ModuleNotFoundError:
    import dotenv
    dotenv.load_dotenv()

In [ ]:
credentials = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey": os.getenv("WATSONX_API_KEY")
}

### Defining the project id
The API requires project id that provides the context for the call. We will obtain the id from the project in which this notebook runs. Otherwise, please provide the project id.

**Hint**: You can find the `project_id` as follows. Open the prompt lab in watsonx.ai. At the very top of the UI, there will be `Projects / <project name> /`. Click on the `<project name>` link. Then get the `project_id` from Project's Manage tab (Project -> Manage -> General -> Details).


In [ ]:
try:
    project_id = os.environ["PROJECT_ID"]
except KeyError:
    project_id = input("Please enter your project_id (hit enter): ")

try:
    space_id = os.environ["SPACE_ID"]
except KeyError:
    space_id = input("Please enter your space id if you have one (hit enter): ") or None

<a id="data"></a>
## Document data loading

Download the file with State of the Union.

In [ ]:
import wget

!mkdir -p 'data'
filename = 'data/state_of_the_union.txt'
url = 'https://raw.github.com/IBM/watson-machine-learning-samples/master/cloud/data/foundation_models/state_of_the_union.txt'

if not os.path.isfile(filename):
    wget.download(url, out=filename)

# LastMile Auto Instrumentation Setup

An instance of `LangChainInstrumentor` is created with a project name. The `instrument()` method is called to instrument the code for tracing and monitoring.

In [ ]:
from tracing_auto_instrumentation.langchain import LangChainInstrumentor

# Create an instance of LangChainInstrumentor and instrument the code
PROJECT_NAME = "ibm x lastmile"
instrumentor = LangChainInstrumentor(project_name=PROJECT_NAME)
instrumentor.instrument()

<a id="build_base"></a>
## Build up knowledge base

The most common approach in RAG is to create dense vector representations of the knowledge base in order to calculate the semantic similarity to a given user query.

In this basic example, we take the State of the Union speech content (filename), split it into chunks, embed it using an open-source embedding model, load it into <a href="https://www.trychroma.com/" target="_blank" rel="noopener no referrer">Chroma</a>, and then query it.

In [ ]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma

loader = TextLoader(filename)
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

The dataset we are using is already split into self-contained passages that can be ingested by Chroma.

### Create an embedding function

Note that you can feed a custom embedding function to be used by chromadb. The performance of Chroma db may differ depending on the embedding model used. In following example we use watsonx.ai Embedding service. We can check available embedding models using `get_embedding_model_specs`

In [ ]:
from ibm_watsonx_ai.foundation_models.utils import get_embedding_model_specs

get_embedding_model_specs(credentials.get('url'))

In [ ]:
from langchain_ibm import WatsonxEmbeddings
from ibm_watsonx_ai.foundation_models.utils.enums import EmbeddingTypes

embeddings = WatsonxEmbeddings(
    model_id=EmbeddingTypes.IBM_SLATE_30M_ENG.value,
    url=credentials["url"],
    apikey=credentials["apikey"],
    project_id=project_id,
    space_id=space_id
)

docsearch = Chroma.from_documents(texts, embeddings)

#### Compatibility watsonx.ai Embeddings with LangChain

 LangChain retrievals use `embed_documents` and `embed_query` under the hood to generate embedding vectors for uploaded documents and user query respectively.

In [ ]:
help(WatsonxEmbeddings)

<a id="models"></a>
## Foundation Models on `watsonx.ai`

IBM watsonx foundation models are among the <a href="https://python.langchain.com/docs/integrations/llms/watsonxllm" target="_blank" rel="noopener no referrer">list of LLM models supported by Langchain</a>. This example shows how to communicate with <a href="https://newsroom.ibm.com/2023-09-28-IBM-Announces-Availability-of-watsonx-Granite-Model-Series,-Client-Protections-for-IBM-watsonx-Models" target="_blank" rel="noopener no referrer">Granite Model Series</a> using <a href="https://python.langchain.com/docs/get_started/introduction" target="_blank" rel="noopener no referrer">Langchain</a>.

### Defining model
You need to specify `model_id` that will be used for inferencing:

In [ ]:
from ibm_watsonx_ai.foundation_models.utils.enums import ModelTypes

model_id = ModelTypes.GRANITE_13B_CHAT_V2

### Defining the model parameters
We need to provide a set of model parameters that will influence the result:

In [ ]:
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from ibm_watsonx_ai.foundation_models.utils.enums import DecodingMethods

parameters = {
    GenParams.DECODING_METHOD: DecodingMethods.GREEDY,
    GenParams.MIN_NEW_TOKENS: 1,
    GenParams.MAX_NEW_TOKENS: 100,
    GenParams.STOP_SEQUENCES: ["<|endoftext|>"]
}

### LangChain CustomLLM wrapper for watsonx model
Initialize the `WatsonxLLM` class from Langchain with defined parameters and `ibm/granite-13b-chat-v2`. 

In [ ]:
from langchain_ibm import WatsonxLLM

watsonx_granite = WatsonxLLM(
    model_id=model_id.value,
    url=credentials.get("url"),
    apikey=credentials.get("apikey"),
    project_id=project_id,
    params=parameters
)

<a id="predict"></a>
## Generate a retrieval-augmented response to a question

Build the `RetrievalQA` (question answering chain) to automate the RAG task.

In [ ]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(llm=watsonx_granite, chain_type="stuff", retriever=docsearch.as_retriever())

### Sample QA chain

Get questions from the previously loaded test dataset.

In [ ]:
query = "What is the goal of the Cancer Moonshot initiative mentioned in the speech?"
qa.invoke(query)

# Evaluations

This section demonstrates how to evaluate the outputs of the question-answering model using the `lastmile_eval` package.


In [ ]:
from lastmile_eval.rag.debugger.api.evaluation import (
    create_input_set,
    run_and_evaluate,
    SaveOptions,
)

# Define inputs and ground truth for the evaluation
inputs = ["What is the goal of the Cancer Moonshot initiative mentioned in the speech?", # 1
          "What did President Zelenskyy say in his speech to the European Parliament?", # 2
          'What statement did the President make regarding the sanctions on Russia?',
          "What did the President say about the future exploration of Mars?"]  # 4 (not in data)

ground_truths = ["The Cancer Moonshot initiative aims to cut the cancer death rate by at least 50% over the next 25 years, turn more cancers into treatable diseases, and provide more support for patients and families.", # 1
                 'In his speech to the European Parliament, President Zelenskyy said, "Light will win over darkness."', # 2
                 'We are cutting off Russia’s largest banks from the international financial system.  Preventing Russia’s central bank from defending the Russian Ruble making Putin’s $630 Billion “war fund” worthless.', #3
                 "The President did not address the future exploration of Mars in the speech."] # 4

evaluator_names = {"qa", "similarity", "relevance"}

In [ ]:
# Run and evaluate outputs using the created input set

# Wrap the qa.invoke function to return the final value as the output
def run_qa_and_return_final_value(query) -> str:
    return qa.invoke(query)["result"]

save_options = SaveOptions(
    example_set_name="State of the Union Example Cases", 
    evaluation_result_name="State of the Union Evaluation Results"
)

create_input_set_separately = False

if create_input_set_separately:
    # This will create an input set, then evaluation set in two separate steps
    # The main difference is that when you create the input (test) set, you aren't 
    # running the `run_query_fn` function, so you won't have the trace data associated
    # with your inputs
    input_set_id = create_input_set(
        project_name=PROJECT_NAME,
        queries=inputs, 
        input_set_name="State of the Union Test Cases",
        ground_truths=ground_truths,
    ).id
    run_and_evaluate(
        project_name=PROJECT_NAME,
        run_query_fn=run_qa_and_return_final_value, # type: ignore
        input_set_id=input_set_id, 
        evaluators=evaluator_names,
        save_options=save_options,
    )
else:
    # An input set hasn't been created yet, so the trace data will be
    # created alongside with it
    run_and_evaluate(
        project_name=PROJECT_NAME,
        run_query_fn=run_qa_and_return_final_value, # type: ignore
        inputs=inputs,
        ground_truths=ground_truths, 
        evaluators=evaluator_names,
        save_options=save_options,
    )

# Launch The Rag Debugger UI

<img width="1915" alt="Screenshot 2024-05-21 at 11 30 01 PM" src="https://github.com/lastmile-ai/aiconfig/assets/141073967/2f2a72c9-fb07-402b-bc6f-fdc46f529edd"> 

<br><br>

<img width="1917" alt="Screenshot 2024-05-21 at 11 30 14 PM" src="https://github.com/lastmile-ai/aiconfig/assets/141073967/4a98b9b3-b96e-430e-9156-79560cbea1be">

In [ ]:
# Launch the rag-debugger
# !rag-debug launch

<a id="summary"></a>
## Summary and next steps

 You successfully completed this notebook!.
 
 You learned how to answer question using RAG using watsonx and LangChain.
 
Check out our _<a href="https://ibm.github.io/watsonx-ai-python-sdk/samples.html" target="_blank" rel="noopener no referrer">Online Documentation</a>_ for more samples, tutorials, documentation, how-tos, and blog posts. 

Copyright © 2023, 2024 IBM. This notebook and its source code are released under the terms of the MIT License.